In [379]:
import pandas as pd
import numpy as np

def delColumn(df, delWith):
    """
    Fonction qui supprimer les colonne comportant le nom donné
    
    df: pandas.dataframe -> data a traité
    delWith: str -> valeur qui permettras d'effectuer la suppression des colonne choisi
    """
    return df[df.columns.drop(list(df.filter(regex=delWith)))]

def cleanerData(df, columns=[], letNan=False):
    """
    Fonction qui a pour role d'éffacer les valeurs erroné dans la dataframe
    
    df: pandas.dataframe -> data a traité
    column: str[] -> selectionne la colonne en particulier
    letNan: bool -> si il est activé toute les valeurs erronés seront null et non remplacé par la moyenne de la colonne
    """
    if columns == []:
        for col in df:
            df[col] = pd.to_numeric(df[col],errors='coerce')
            df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)
            if not letNan :
                mean = df[col].mean()
                df[col] = df[col].apply(lambda x: mean if x == 0 or np.isnan(x) else x)
    else:
        for col in columns:
            df[col] = pd.to_numeric(df[col],errors='coerce')
            df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)
            if not letNan:
                mean = df[col].mean()
                df[col] = df[col].apply(lambda x: mean if x == 0 or np.isnan(x) else x)
    return df

def decribeData(df, roundValue=-1, nbNan=False, selectColumns=[], mode=""):
    """
    Fonction pour generer une description de la data selon differente option
    
    df: pandas.dataframe -> data a traité
    roundValue: int -> permet de recuperer les valeur a l'arrondi
    nbNan: bool -> pour compter le nombre de valeur null
    selectColumns: str[] -> pour traiter des colonne specifique
    mode: str -> mode possible; mean, std, sum, var, min, max, median, info, unique, type:int[] liste des quantiles recherché
    """
    described = df
    if selectColumns != []:
        described = df.loc[:,selectColumns]
    
    if nbNan:
        if selectColumns != []:
            print('\033[93m' + "SELECTCOLUMNS Warn:"+ '\033[95m' +" si nbNan est actif SELECTCOLUMNS ne sert a rien")
        if mode != "":
            print('\033[93m' + "MODE Warn:"+ '\033[95m' +" si nbNan est actif MODE ne sert a rien")
        if roundValue != -1:
            print('\033[93m' + "ROUNDVALUE Warn:"+ '\033[95m' +" si nbNan est actif ROUNDVALUE ne sert a rien")
        return described.isnull().sum()
    
    if mode == "mean":
        described = described.mean()
    elif mode == "std":
        described =  described.std()
    elif mode == "sum":
        described =  described.sum()
    elif mode == "var":
        described =  described.var()
    elif mode == "min":
        described =  described.min()
    elif mode == "max":
        described =  described.max()
    elif mode == "median":
        described =  described.median()
    elif mode == "info":
        described =  described.info()
    elif mode == "unique":
        liste = {}
        for col in described:
            print(col,"have",len(described[col].unique()),"unique value")
            liste[col] = described[col].unique()
        return liste
    elif type(mode) == list:
        return described.quantile(mode)
    
    if roundValue != -1:
        described = described.round(roundValue)
    if mode != "":
        return described
    
    described = described.describe()
    if roundValue != -1:
        described = described.round(roundValue)
    
    return described

def select(df, column, condition=()):
    """
    selectionne les donnée d'une colonne, il y a la posibilité de selectionneé une colone selon une condition
    
    df: pandas.dataframe -> data a traité
    column: str -> selectionne la colonne en particulier
    condition: (str, int | float) -> condition possible; <, >, <=, >=, != et ==
    """
    if condition != () and len(condition) == 2:
        mode, form = condition
        if mode == ">":
            result = df[column] > form
            return df.loc[result]
        elif mode == "<":
            result = df[column] > form
            return df.loc[result]
        elif mode == "!=":
            result = df[column] != form
            return df.loc[result]
        elif mode == "==":
            result = df[column] == form
            return df.loc[result]
        elif mode == "<=":
            result = df[column] <= form
            return df.loc[result]
        elif mode == ">=":
            result = df[column] >= form
            return df.loc[result]
    else:
        return df[column]
    
def changeValue(df, columns, origin, new, condition=()):
    """
    change la ou les valeurs donnée dans origin par new dans le tableau 
    
    df: pandas.dataframe -> data a traité
    column: str[] -> selectionne la colonne en particulier
    origin: object -> object d'origine a modifier, si la condition est active elle ne sera pas pris en compte, il peut etre initialisé a ""
    new: object -> la nouvelle valeur qui sera assigné selon les differente possibilité
    condition (str, object): assigne la valeur new si la condition est respecteer sinon elle garde la valeurs de  base
        origin ne sera pas pris en compte, il peut etre initialisé a ""
    condition ("around", int) : aroundi la valeur selon les colonnes selectionné, int correspond du nombre de chiffre apres la virgule
    """
    for i in range(len(columns)):
        if condition != () and len(condition) == 2:
            mode, form = condition
            if mode == ">":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x > form else x)
            elif mode == "<":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x < form else x)
            elif mode == "!=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x != form else x)
            elif mode == "==":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x == form else x)
            elif mode == "<=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x <= form else x)
            elif mode == ">=":
                df[columns[i]] = df[columns[i]].apply(lambda x: new if x >= form else x)
            elif mode == "around":
                df[columns[i]] = df[columns[i]].apply(lambda x: round(x, form) if type(x) is float else x)
        else:
            df[columns[i]] = df[columns[i]].apply(lambda x: new if x == origin else x)
            
def find(df,column,what, groupby="", sort=""):
    """
    Fonction pour trouver une ou plusieur valeurs dans le tableau en fonction de la colonne

    df: pandas.dataframe -> data a traité
    column: str column selectionné
    what: str[] -> liste des valeurs a chercher
    groupby: str -> rassemblement par; mean, sum, first, last, size
    sort: str -> triage de facon; ascending, descending
    """
    tmp = df[df[column].isin(what)]
    
    if groupby=="mean":
        tmp = tmp.groupby([column]).mean()
    elif groupby=="sum":
        tmp = tmp.groupby([column]).sum()
    elif  groupby=="first":
        tmp = tmp.groupby([column]).first()
    elif  groupby=="last":
        tmp = tmp.groupby([column]).last()
    elif  groupby=="size":
        tmp = tmp.groupby([column]).size()
    
    if sort != "" and groupby != "":
        print('\033[93m' + "SORT Warn:"+ '\033[95m' +" inutile de trier les objets groupé")
        sort=""
    
    if sort=="ascending":
        tmp = tmp.sort_values(by=column, ascending=True)
    elif sort=="descending":
        tmp = tmp.sort_values(by=column, ascending=False)
            
    return tmp
                

In [361]:
df = pd.read_csv('Dataset_6_quality.csv',delimiter="|")

In [362]:
decribeData(df, nbNan=True)

Unnamed: 0                 0
Unnamed: 0.1               0
Unnamed: 0.1.1             0
Unnamed: 0.1.1.1           0
Unnamed: 0.1.1.1.1         0
Unnamed: 0.1.1.1.1.1       0
Unnamed: 0.1.1.1.1.1.1     0
fixed acidity              0
volatile acidity          12
citric acid               15
residual sugar             0
chlorides                  0
free sulfur dioxide       50
total sulfur dioxide       0
density                    0
pH                         0
sulphates                  0
alcohol                    0
quality                    0
dtype: int64

In [363]:
df = delColumn(df,"Unnamed:")
df = cleanerData(df, letNan=True)

In [364]:
decribeData(df, nbNan=True, mode="sum")

MODE Warn: si nbNan est actif MODE ne sert a rien


fixed acidity            77
volatile acidity        148
citric acid              94
residual sugar           65
chlorides                15
free sulfur dioxide      65
total sulfur dioxide      0
density                   0
pH                        0
sulphates                 0
alcohol                   0
quality                   0
dtype: int64

In [365]:
decribeData(df, selectColumns=["pH","alcohol","quality"], mode="max")

pH          4.01
alcohol    14.90
quality     8.00
dtype: float64

In [366]:
select(df, "pH", condition=(">", 4))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1316,5.555556e+10,0.74,0.0,NaN,0.041,16.0,46.0,0.99258,4.01,0.59,12.5,6
1321,5.000000e+00,0.74,0.0,1.2,0.041,16.0,46.0,0.99258,4.01,0.59,12.5,6


In [367]:
tmp = select(df, "alcohol", condition=(">", 4))
decribeData(tmp, roundValue=1, mode="mean", selectColumns=["pH","alcohol","quality"])

pH          3.3
alcohol    10.4
quality     5.6
dtype: float64

In [368]:
decribeData(tmp, roundValue=3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1.522000e+03,1.451000e+03,1505.000,1534.000,1584.000,1534.000,1599.000,1599.000,1599.000,1599.000,1599.000,1599.000
mean,5.475252e+08,5.743166e+08,0.273,2.523,0.087,15.879,46.468,0.997,3.311,0.658,10.423,5.636
std,5.489816e+09,5.621247e+09,0.195,1.370,0.047,10.389,32.895,0.002,0.154,0.170,1.066,0.808
min,0.000000e+00,1.200000e-01,0.000,0.900,0.012,1.000,6.000,0.990,2.740,0.330,8.400,3.000
25%,7.100000e+00,4.000000e-01,0.100,1.900,0.070,7.000,22.000,0.996,3.210,0.550,9.500,5.000
50%,7.900000e+00,5.200000e-01,0.260,2.200,0.079,14.000,38.000,0.997,3.310,0.620,10.200,6.000
75%,9.300000e+00,6.400000e-01,0.430,2.600,0.090,21.000,62.000,0.998,3.400,0.730,11.100,6.000
max,5.555556e+10,5.555556e+10,1.000,15.400,0.611,72.000,289.000,1.004,4.010,2.000,14.900,8.000


In [374]:
decribeData(df, selectColumns=["free sulfur dioxide","total sulfur dioxide"], mode="unique")

free sulfur dioxide have 60 unique value
total sulfur dioxide have 144 unique value


{'free sulfur dioxide': array([11. , 25. , 15. , 17. ,  nan,  9. , 16. , 52. , 51. , 35. ,  6. ,
        29. , 23. , 10. , 21. ,  4. , 14. ,  8. , 22. , 40. , 13. ,  5. ,
         3. ,  7. , 12. , 30. , 33. , 50. , 19. , 20. , 27. , 18. , 28. ,
        34. , 42. , 41. , 37. , 32. , 36. , 24. , 26. , 39. , 40.5, 68. ,
        31. , 38. , 47. ,  1. , 54. , 46. , 45. ,  2. ,  5.5, 53. , 37.5,
        57. , 48. , 43. , 72. , 55. ]),
 'total sulfur dioxide': array([ 34. ,  67. ,  54. ,  60. ,  40. ,  59. ,  21. ,  18. , 102. ,
         65. ,  29. , 145. , 148. , 103. ,  56. ,  71. ,  37. ,  23. ,
         11. ,  35. ,  16. ,  82. , 113. ,  83. ,  50. ,  15. ,  30. ,
         19. ,  87. ,  46. ,  14. , 114. ,  12. ,  96. , 119. ,  73. ,
         45. ,  10. , 110. ,  52. , 112. ,  39. ,  27. ,  94. ,  43. ,
         42. ,  80. ,  51. ,  61. , 136. ,  31. , 125. ,  24. , 140. ,
        133. ,  85. , 106. ,  22. ,  36. ,  69. ,  64. , 153. ,  47. ,
        108. , 111. ,  62. ,  28. ,  89. ,  13

In [378]:
find(df, "quality", [0,1,2,3,4,5,6,7,8,9,10], groupby="size")

quality
3     10
4     53
5    681
6    638
7    199
8     18
dtype: int64

In [373]:
find(df, "quality", [3], sort="ascending")

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
459,0.0,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.00,3
517,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.40,3
690,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.70,3
832,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.90,3
899,8.3,1.020,0.02,3.40,0.084,6.0,11.0,0.99892,3.48,0.49,11.00,3
1299,7.6,1.580,0.00,2.10,0.137,5.0,9.0,0.99476,3.50,0.40,10.90,3
1374,6.8,NaN,0.00,1.20,0.267,16.0,29.0,0.99471,3.32,0.51,9.80,3
1469,7.3,0.980,0.05,2.10,0.061,20.0,49.0,0.99705,3.31,0.55,9.70,3
1478,NaN,0.875,0.05,5.70,0.082,3.0,14.0,0.99808,3.40,0.52,10.20,3
1505,6.7,0.760,0.02,1.80,0.078,6.0,12.0,0.99600,3.55,0.63,9.95,3
